## Import library yang digunakan

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

## Upload Dataset

In [ ]:
# Memuat dataset CSV
df = pd.read_csv("DataDestinasi.csv")

print(df.head())

## Kodingan mengubah column price pada dataset
jika dibawah harga di bawah 20000 akan dilakukan mix dengan range, [5000, 10000, 15000, 20000]

In [ ]:
df['Price'] = pd.to_numeric(df['Price'], errors='coerce').fillna(0).astype(int)

print("Jumlah nilai 0 pada kolom 'Price' sebelum diganti:", (df['Price'] <= 20000).sum())

replacement_values = [5000, 10000, 15000, 20000]

df['Price'] = df['Price'].apply(lambda x: np.random.choice(replacement_values) if x == 0 else x)

print("Jumlah nilai 0 pada kolom 'Price' setelah diganti:", (df['Price'] <= 20000).sum())

Jumlah nilai 0 pada kolom 'Price' sebelum diganti: 168
Jumlah nilai 0 pada kolom 'Price' setelah diganti: 0


## Tranform pada column "Price", "Place_Name"
Dan menggunakan 2 layers dengan hyperparameter 128, 64
dengan aktivation softmax pada layers output

In [ ]:
scaler = StandardScaler()
df['Price_scaled'] = scaler.fit_transform(df[['Price']])

label_encoder = LabelEncoder()
df['Place_Label'] = label_encoder.fit_transform(df['Place_Name'])

city_encoder = OneHotEncoder(sparse_output=False)
city_encoded = city_encoder.fit_transform(df[['City']])

X_price_city = np.hstack([df['Price_scaled'].values.reshape(-1, 1), city_encoded])

place_labels = df['Place_Label'].values

model = models.Sequential([
    layers.InputLayer(shape=(X_price_city.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(df['Place_Name'].unique()), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_price_city, place_labels, epochs=20, batch_size=32)

model.save("model_destinasi_wisata.h5")





Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0000e+00 - loss: 6.3340
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 9.1996e-04 - loss: 6.3136
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0077 - loss: 6.2664
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0234 - loss: 6.1707
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0155 - loss: 5.9984
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0344 - loss: 5.6654
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0259 - loss: 5.2804
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0448 - loss: 4.8542
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0362 - loss: 4.5815
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0483 - loss: 4.3462
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0285 - loss: 4.1873
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0

## Membuat function recommend_places

In [ ]:
def recommend_places(price, city, top_n=5):
    price_scaled = scaler.transform(np.array([[price]]))
    city_encoded = city_encoder.transform(np.array([[city]]))
    user_input = np.hstack([price_scaled, city_encoded])

    probabilities = model.predict(user_input)[0]

    top_indices = np.argsort(probabilities)[-top_n:][::-1]

    recommended_places = label_encoder.inverse_transform(top_indices)
    return recommended_places


## Membuat alternative function recommended_tour_guid

In [ ]:
from itertools import combinations

# Fungsi untuk merekomendasikan destinasi berdasarkan budget
def recommend_tour_guide(budget, city):
    # Filter destinasi berdasarkan kota
    city_places = df[df['City'] == city]

    # Mengambil data nama dan harga
    places = list(zip(city_places['Place_Name'], city_places['Price']))

    best_combination = []
    max_cost = 0

    # Menggunakan kombinasi untuk mencari 3 destinasi
    for combo in combinations(places, 3):
        total_cost = sum(price for _, price in combo)
        if total_cost <= budget and total_cost > max_cost:
            max_cost = total_cost
            best_combination = combo

    return best_combination, max_cost

Contoh penggunaan function

In [ ]:
# Contoh penggunaan
user_budget = 100000  # Anggaran pengguna
user_city = 'Bekasi'  # Kota yang diminta pengguna
recommended_places = recommend_places(user_budget, user_city)

print(f"Rekomendasi untuk anggaran {user_budget} di kota {user_city}:")
for i, place in enumerate(recommended_places, 1):
    print(f"{i}. {place}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Rekomendasi untuk anggaran 100000 di kota Bekasi:
1. Trans Snow World Juanda
2. Go! Wet Waterpark
3. Waterboom Lippo Cikarang
4. Mountain View Golf Club
5. Trans Studio Bandung


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [ ]:
# Contoh penggunaan
user_budget = 30000  # Anggaran pengguna
user_city = 'Cilegon'  # Kota yang diminta pengguna
recommended_places, total_cost = recommend_tour_guide(user_budget, user_city)

print(f"Rekomendasi untuk anggaran {user_budget} di kota {user_city}:")
for i, (place, price) in enumerate(recommended_places, 1):
    print(f"{i}. {place} - Harga: {price}")
print(f"Total biaya: {total_cost}")

Rekomendasi untuk anggaran 30000 di kota Cilegon:
1. Pantai Batu Saung - Harga: 10000
2. Bukit Kembang Kuning - Harga: 5000
3. Masjid Agung Cilegon - Harga: 15000
Total biaya: 30000


## Save model

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('model_destinasi_wisata.h5')
model.predict()

In [ ]:
!pip install tensorflowjs

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 84.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      

In [ ]:
!tensorflowjs_converter --input_format=keras model_destinasi_wisata.h5 ./model_directory


2024-11-20 15:25:48.445791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732116348.488979   26370 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732116348.502085   26370 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file


In [ ]:
!tensorflowjs_converter --input_format=keras model_destinasi_wisata.h5 "./"

2024-11-20 10:42:01.470944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732099321.494818   10355 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732099321.502472   10355 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file


In [ ]:
import numpy as np
from itertools import combinations

# Fungsi untuk merekomendasikan destinasi berdasarkan budget
def recommend_tour_guide(budget, city):
    # Filter destinasi berdasarkan kota
    city_places = df[df['City'] == city]

    # Mengambil data nama dan harga
    places = list(zip(city_places['Place_Name'], city_places['Price']))
    place_labels = city_places['Place_Label'].values

    # Siapkan input untuk model
    price_scaled = scaler.transform(np.array([[budget]]))
    city_encoded = city_encoder.transform(np.array([[city]]))
    user_input = np.hstack([price_scaled, city_encoded])

    # Dapatkan probabilitas dari model
    probabilities = model.predict(user_input)[0]

    # Menggabungkan nama, harga, dan probabilitas
    scored_places = list(zip(places, probabilities))

    # Mengurutkan berdasarkan probabilitas
    scored_places.sort(key=lambda x: x[1], reverse=True)

    best_combination = []
    max_cost = 0

    # Menggunakan kombinasi untuk mencari 3 destinasi
    for combo in combinations(scored_places, 3):
        total_cost = sum(price for (_, price), _ in combo)
        if total_cost <= budget and total_cost > max_cost:
            max_cost = total_cost
            best_combination = combo

    return [(place[0][0], place[0][1]) for place in best_combination], max_cost